In [5]:
#2018 HMDA Edit Testing File Generator
from collections import OrderedDict
from io import StringIO
import json
import os
import pandas as pd
import random

#custom imports
import lar_constraints
import lar_generator
from rules_engine import rules_engine
from test_file_generator import test_data
import utils
!pwd

/Users/roellk/hmda/hmda-test-files/python


In [6]:
#2018 Filing Instruction Guide: https://www.consumerfinance.gov/data-research/hmda/static/for-filers/2018/2018-HMDA-FIG.pdf

use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
             'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependencies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
                    header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)

In [7]:
#load schemas for LAR and transmittal sheet
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))

In [9]:
lar_gen = lar_generator.lar_gen(lar_schema_df, ts_schema_df, counties=counties, tracts=tracts) #instantiate generator
lar_const = lar_constraints.lar_constraints(counties=counties, tracts=tracts)#instantiate constraints
lar_validator = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, cbsa_data=cbsas)

In [10]:
#Set parameters for data creation
file_length = 5 #set number of rows in test file
first = True
lei = None

In [11]:
#Create a sample TS row
#set dummy values for TS row
def make_ts_row(file_length, lei):
    ts_row = OrderedDict()
    ts_row["record_id"]="1"
    ts_row["inst_name"]="Ficus Bank"
    ts_row["calendar_year"]= "2018"
    ts_row["calendar_quarter"]="4"
    ts_row["contact_name"]="Mr. Smug Pockets"
    ts_row["contact_tel"]="555-555-5555"
    ts_row["contact_email"]="pockets@ficus.com"
    ts_row["contact_street_address"]="1234 Ficus Lane"
    ts_row["office_city"]="Ficusville"
    ts_row["office_state"]="UT"
    ts_row["office_zip"]="84096"
    ts_row["federal_agency"]="9"
    ts_row["lar_entries"]= str(file_length)
    ts_row["tax_id"]="01-0123456"
    ts_row["lei"]=str(lei)
    return ts_row

In [14]:
def get_const_list():
    """Creates a list of constraints from the functions in the lar_constraints object."""
    constraints = [] 
    for func in dir(lar_const):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            constraints.append(func)
    return constraints
            
def constraints_loop(constraints=[], row=None, row_base=None):
    for const in constraints:
        row = apply_constraint(row, const)
        diff = get_diff(row, row_base)
    return row

def apply_constraint(row, func):
    """Applies all constraints in the constrains list and returns a LAR row in dictionary format."""
    row_start = row.copy()
    row = getattr(lar_const, func)(row) #apply constraint to row
    diff_1, diff_2 = get_diff(row, row_start)
    if len(diff_1) > 0:
        print(str(func))
        print(diff_1, "\n\n", diff_2)
    return row

def get_diff(row, row_base):
    """Checks the difference between an initial row and the row after constraints are applied"""
    initial_row = set(row_base.items()) #convert initial row to set
    changed_row = set(row.items()) #convert constrained row to set
    diff_1 = (changed_row - initial_row) #subtract row sets to show changes from constraint funcs
    diff_2 = (initial_row - changed_row)
    return diff_1, diff_2

def validation(row, ts_row):
    """"""
    lar_data = pd.DataFrame(row, index=[1])
    ts_data = pd.DataFrame(ts_row, index=[0])
    rules_check = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, cbsa_data=cbsas) #instantiate edits rules engine
    rules_check.load_lar_data(lar_data)
    rules_check.load_ts_data(ts_data)
    for func in dir(rules_check):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            #print("applying:", func)
            getattr(rules_check, func)()
    return rules_check.results

def write_file(ts_input=None, lar_input=None, directory="../edits_files/", name="passes_all.txt"):
    """Takes a TS row as a dictionary and LAR data as a dataframe. Writes LAR data to file and 
    re-reads it to combine with TS data to make a full file."""
    #make directories for files if they do not exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    #write LAR dataframe to file
    parts_dir = directory+"file_parts/"
    if not os.path.exists(parts_dir):
        os.makedirs(parts_dir)
        
    lar_input.to_csv(parts_dir + "lar_data.txt", sep="|", header=False, index=False, index_label=False)
    #load LAR data as file rows
    with open(parts_dir + "lar_data.txt", 'r') as lar_data:
        lar = lar_data.readlines()

    with open(directory + name, 'w') as final_file:
        final_file.write("|".join(ts_input.values())+"\n")
        for line in lar:
            final_file.write("{line}".format(line=line))

In [15]:
first = True
for i in range(0, file_length): #loop over file length
    print("making new row {row_num}\n\n".format(row_num=i), "*"*50)
    if lei:
        row = lar_gen.make_row(lei=lei) #generate new LEI. The same LEI must be used for each row
    else:
        row = lar_gen.make_row() #create new row
        
    lei = row["lei"] #copy LEI from previous row
    iters = 1 #start iteration count for checking diff time

    ts_row = make_ts_row(file_length, lei) #create dictionary of TS row fields
    stop = False
    #flag for starting the LAR dataframe
    while stop == False:
        row_base = row.copy() #copy row to enable diff
        res = pd.DataFrame(validation(row, ts_row))
        print(res[res.status=="failed"])
        if len(res[res.status=="failed"])<=0:
            stop = True
        else:
            print("\nstarting constraints iteration {iter}".format(iter=iters))
            row = constraints_loop(get_const_list(), row, row_base)
        iters+=1
    
    if first: #create first row of dataframe
        lar_frame = pd.DataFrame(row, index=[1])
        first = False
        print("finished row\n")
    else: #add additional rows to dataframe
        #print("concating")
        print("finished row\n")
        new_lar = pd.DataFrame(row, index=[1])
        lar_frame = pd.concat([lar_frame, new_lar], axis=0)
        
#lar_frame.reset_index(inplace=True) #reset index
#lar_frame.drop("index", inplace=True, axis=1) #drop additional index column
print("LAR dataframe complete")

making new row 0

 **************************************************
    edit_name  fail_count                      fields  \
24     v612_2         1.0                loan_purpose   
29     v614_1         1.0                 preapproval   
34     v615_2         1.0                const_method   
41     v619_3         1.0                 action_date   
58     v629_2         1.0         app ethnicity basis   
64     v631_3         1.0          Co-App Ethnicities   
68     v632_3         1.0      Co-App Ethnicity Basis   
77     v636_2         1.0        Applicant Race Basis   
86     v639_2         1.0     Co-Applicant Race Basis   
93     v643_1         1.0         Applicant Sex Basis   
117    v656_2         1.0           Type of Purchaser   
119    v657_2         1.0                 Rate Spread   
122    v658_2         1.0                       HOEPA   
127    v662_1         1.0              App Score Name   
128    v662_2         1.0              App Score Name   
135    v667_1     

    edit_name  fail_count              fields  \
75     v635_4         1.0     Applicant Races   
84     v638_4         1.0  Co-Applicant Races   
142    v669_4         1.0  Denial Reasons 1-4   

                                             row_ids row_type  status  
75   [LR9GFFFC5Y0UCVR2PE3M2WDBMNU74FHLWFKP2XMSEIB27]      LAR  failed  
84   [LR9GFFFC5Y0UCVR2PE3M2WDBMNU74FHLWFKP2XMSEIB27]      LAR  failed  
142  [LR9GFFFC5Y0UCVR2PE3M2WDBMNU74FHLWFKP2XMSEIB27]      LAR  failed  

starting constraints iteration 2
v635_4_const
{('app_race_5', ''), ('app_race_2', ''), ('app_race_3', ''), ('app_race_4', '')} 

 {('app_race_3', '44'), ('app_race_2', '21'), ('app_race_5', '41'), ('app_race_4', '26')}
v638_const
{('co_app_race_4', ''), ('co_app_race_2', ''), ('co_app_race_5', ''), ('co_app_race_3', '')} 

 {('co_app_race_2', '27'), ('co_app_race_4', '5'), ('co_app_race_5', '22'), ('co_app_race_3', '23')}
v669_const
{('denial_4', ''), ('denial_2', ''), ('denial_3', '')} 

 {('denial_3', '7'),

    edit_name  fail_count              fields  \
142    v669_4         1.0  Denial Reasons 1-4   

                                             row_ids row_type  status  
142  [LR9GFFFC5Y0UCVR2PE3MUIYZ24ETCZ0LPSK73CF001A29]      LAR  failed  

starting constraints iteration 2
v669_const
{('denial_4', ''), ('denial_2', ''), ('denial_3', '')} 

 {('denial_3', '7'), ('denial_2', '9'), ('denial_4', '3')}
v671_const
{('denial_code_9', '')} 

 {('denial_code_9', '7WVLZ5BCLQFCLR0KJBZ91KKAM60MWYR249S2753AWE93WZKQTDD5FX3P6JUEM5NOGOTGZ6X7YZVKCWV5J6Y2O5Y0DR64T53UP9DVUG4O460QE5N0T0XGIB0F66OAAL0J')}
Empty DataFrame
Columns: [edit_name, fields, row_type, status]
Index: []
finished row

making new row 3

 **************************************************
    edit_name  fail_count                      fields  \
24     v612_2         1.0                loan_purpose   
29     v614_1         1.0                 preapproval   
30     v614_2         1.0                 preapproval   
41     v619_3        

    edit_name  fail_count                   fields  \
21     v610_2         1.0                 app_date   
65     v631_4         1.0       Co-App Ethnicities   
74     v635_3         1.0          Applicant Races   
84     v638_4         1.0       Co-Applicant Races   
142    v669_4         1.0       Denial Reasons 1-4   
215    v696_3         1.0  AUS Systems and Results   
219    v700_1         1.0          AUS and Results   
220    v700_2         1.0          AUS and Results   
221      v701         1.0          AUS and Results   

                                             row_ids row_type  status  
21   [LR9GFFFC5Y0UCVR2PE3MVK154DRAZ3KJESHXTZCI2HO03]      LAR  failed  
65   [LR9GFFFC5Y0UCVR2PE3MVK154DRAZ3KJESHXTZCI2HO03]      LAR  failed  
74   [LR9GFFFC5Y0UCVR2PE3MVK154DRAZ3KJESHXTZCI2HO03]      LAR  failed  
84   [LR9GFFFC5Y0UCVR2PE3MVK154DRAZ3KJESHXTZCI2HO03]      LAR  failed  
142  [LR9GFFFC5Y0UCVR2PE3MVK154DRAZ3KJESHXTZCI2HO03]      LAR  failed  
215  [LR9GFFFC5Y0UCVR2PE3MV

In [17]:
#check validity and syntax of data using rules_engine
#instantiate edits rules engine
validator = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, cbsa_data=cbsas) 
#load data to validator engine
validator.load_lar_data(lar_frame)
validator.load_ts_data(pd.DataFrame(ts_row, index=[0], columns=validator.ts_field_names))
#check data
for func in dir(validator):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(validator, func)()

#get validation results
results_df = pd.DataFrame(validator.results)
results_df[results_df.status=="failed"]

,edit_name,fields,row_type,status


In [18]:
#Quality and Macro field interrelationship constraints:

In [19]:
#write sample file to disk
write_file(ts_input=ts_row, lar_input=lar_frame, name="passes_10_rows.txt") #writes created file to disk
lar_frame

,record_id,lei,uli,app_date,loan_type,loan_purpose,preapproval,const_method,occ_type,loan_amount,...,aus_code_5,aus_result_1,aus_result_2,aus_result_3,aus_result_4,aus_result_5,aus_code_16,reverse_mortgage,open_end_credit,business_purpose
1,2,LR9GFFFC5Y0UCVR2PE3M,4N728065GEU3JR8QZLKQW0,NA,4,2,2,2,1,25593,...,,17,,,,,,2,2,-1
1,2,LR9GFFFC5Y0UCVR2PE3M,LR9GFFFC5Y0UCVR2PE3M2WDBMNU74FHLWFKP2XMSEIB27,20180510,2,5,2,2,1,329220,...,,11,,11,2,,,2,-1,1
1,2,LR9GFFFC5Y0UCVR2PE3M,LR9GFFFC5Y0UCVR2PE3MUIYZ24ETCZ0LPSK73CF001A29,20181212,1,1,2,2,3,264638,...,LP8FAB85CWEKNMU6CT9ATY4J8KV5DSCZKSLZ4U00VWC636...,15,,4,5,5,,2,2,2
1,2,LR9GFFFC5Y0UCVR2PE3M,LR9GFFFC5Y0UCVR2PE3MVK154DRAZ3KJESHXTZCI2HO03,NA,1,5,2,2,2,1947,...,,17,,,,,,2,-1,2
1,2,LR9GFFFC5Y0UCVR2PE3M,9TLFWE9GBVU5WN0FEVNGBX,NA,2,31,2,2,1,276057,...,,17,,,,,,2,-1,2


In [ ]:
file_maker = test_data(ts_schema_df, lar_schema_df)
ts_data, lar_data = utils.read_data_file(path="../edits_files/", data_file="passes_10_rows.txt")
file_maker.load_data_frames(ts_data, lar_data)
file_maker.s300_1_file()
file_maker.s300_2_file()

In [ ]:
validator.read_data_file(path="../edits_files/syntax/", data_file="s300_2.txt")
validator.reset_results()
#check data
for func in dir(validator):
    if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(validator, func)()

#get validation results
results_df = pd.DataFrame(validator.results)
results_df[results_df.status=="failed"]

In [22]:
for col in lar_frame.columns:
    print(col+",")

record_id,
lei,
uli,
app_date,
loan_type,
loan_purpose,
preapproval,
const_method,
occ_type,
loan_amount,
action_taken,
action_date,
street_address,
city,
state,
zip_code,
county,
tract,
app_eth_1,
app_eth_2,
app_eth_3,
app_eth_4,
app_eth_5,
app_eth_free,
co_app_eth_1,
co_app_eth_2,
co_app_eth_3,
co_app_eth_4,
co_app_eth_5,
co_app_eth_free,
app_eth_basis,
co_app_eth_basis,
app_race_1,
app_race_2,
app_race_3,
app_race_4,
app_race_5,
app_race_native_text,
app_race_asian_text,
app_race_islander_text,
co_app_race_1,
co_app_race_2,
co_app_race_3,
co_app_race_4,
co_app_race_5,
co_app_race_native_text,
co_app_race_asian_text,
co_app_race_islander_text,
app_race_basis,
co_app_race_basis,
app_sex,
co_app_sex,
app_sex_basis,
co_app_sex_basis,
app_age,
co_app_age,
income,
purchaser_type,
rate_spread,
hoepa,
lien,
app_credit_score,
co_app_credit_score,
app_score_name,
app_score_code_8,
co_app_score_name,
co_app_score_code_8,
denial_1,
denial_2,
denial_3,
denial_4,
denial_code_9,
loan_costs,
points

In [25]:
ts_row.keys()

odict_keys(['record_id', 'inst_name', 'calendar_year', 'calendar_quarter', 'contact_name', 'contact_tel', 'contact_email', 'contact_street_address', 'office_city', 'office_state', 'office_zip', 'federal_agency', 'lar_entries', 'tax_id', 'lei'])

In [ ]:
lar_header = lar_frame.columns